<a href="https://colab.research.google.com/github/nmtan2001/Bassa/blob/master/Music_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Music Recommendation System

Ngo Minh Tan - For CS50X

16/10/23

Moscow - Russia




**INTRODUCTION**

Music recommendation systems are used to suggest songs or playlists to users based on their listening history or preferences. They are a popular type of recommendation system, and are used by many music streaming services, such as Spotify and Apple Music.

A typical music recommendation system using machine learning in the real world works in the following steps:

1. Data collection: The system collects data on users's listening habits, such as the songs they play, the playlists they create, and the artists they follow.
2. Feature extraction: The system extracts features from the music data, such as the genre, tempo, rhythm, and mood of the songs.
3. Model building: The system uses a machine learning algorithm to build a model that can predict which songs a user is likely to like, based on their listening history and the features of the music.
4. Recommendation generation: The system uses the model to generate a list of recommended songs for the user.

As for the sake of the final project/educational purpose, the implementation will be simplied from how the system actual work.





**IMPLEMENTATION**

Based on the project, there are serveral questions we need to answer


* What's the goal of our system and where we'll take the data? (recommend base on real playlists provided from APIs like Spotipy, recommend base on a datasets, recommend base on the sound provided...)
* How we'll approach this problem (using the lyrics, using the metadata/characteristics of the songs,...)
* What algorithms we'll use(TF-IDF, Collaborative filtering, Content-base filtering, Hybrid filtering, Clustering...)

After considering, I decided that I'll take data from a dataset, instead of the playlist-id approach because it's more fit for the project. For the algorithm, I will use cosine similarity

#EXPLORE DATASET

The dataset I chose is: https://www.kaggle.com/datasets/jashanjeetsinghhh/songs-dataset

In [ ]:
#Import library
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split


import warnings
warnings.filterwarnings('ignore')

In [ ]:
pip install spotipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.3/250.3 kB 5.8 MB/s eta 0:00:00


In [ ]:
import requests
import base64

# Replace with your own Client ID and Client Secret
CLIENT_ID = '28339b556a91431ea6ea47939c150dce'
CLIENT_SECRET = '083b618b22be444f9348068f60d190a9'

# Base64 encode the client ID and client secret
client_credentials = f"{CLIENT_ID}:{CLIENT_SECRET}"
client_credentials_base64 = base64.b64encode(client_credentials.encode())

# Request the access token
token_url = 'https://accounts.spotify.com/api/token'
headers = {
    'Authorization': f'Basic {client_credentials_base64.decode()}'
}
data = {
    'grant_type': 'client_credentials'
}
response = requests.post(token_url, data=data, headers=headers)

if response.status_code == 200:
    access_token = response.json()['access_token']
    print("Access token obtained successfully.")
else:
    print("Error obtaining access token.")
    exit()

Access token obtained successfully.


In [ ]:
df = pd.read_csv('dataset.csv')
df.head(10)

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
0,0,5SuOikwiRyPMVoIQDJUgSV,Gen Hoshino,Comedy,Comedy,73,230666,False,0.676,0.4610,...,-6.746,0,0.1430,0.0322,0.000001,0.3580,0.7150,87.917,4,acoustic
1,1,4qPNDBW1i3p13qLCt0Ki3A,Ben Woodward,Ghost (Acoustic),Ghost - Acoustic,55,149610,False,0.420,0.1660,...,-17.235,1,0.0763,0.9240,0.000006,0.1010,0.2670,77.489,4,acoustic
2,2,1iJBSr7s7jYXzM8EGcbK5b,Ingrid Michaelson;ZAYN,To Begin Again,To Begin Again,57,210826,False,0.438,0.3590,...,-9.734,1,0.0557,0.2100,0.000000,0.1170,0.1200,76.332,4,acoustic
3,3,6lfxq3CG4xtTiEg7opyCyx,Kina Grannis,Crazy Rich Asians (Original Motion Picture Sou...,Can't Help Falling In Love,71,201933,False,0.266,0.0596,...,-18.515,1,0.0363,0.9050,0.000071,0.1320,0.1430,181.740,3,acoustic
4,4,5vjLSffimiIP26QG5WcN2K,Chord Overstreet,Hold On,Hold On,82,198853,False,0.618,0.4430,...,-9.681,1,0.0526,0.4690,0.000000,0.0829,0.1670,119.949,4,acoustic
5,5,01MVOl9KtVTNfFiBU9I7dc,Tyrone Wells,Days I Will Remember,Days I Will Remember,58,214240,False,0.688,0.4810,...,-8.807,1,0.1050,0.2890,0.000000,0.1890,0.6660,98.017,4,acoustic
6,6,6Vc5wAMmXdKIAM7WUoEb7N,A Great Big World;Christina Aguilera,Is There Anybody Out There?,Say Something,74,229400,False,0.407,0.1470,...,-8.822,1,0.0355,0.8570,0.000003,0.0913,0.0765,141.284,3,acoustic
7,7,1EzrEOXmMH3G43AXT1y7pA,Jason Mraz,We Sing. We Dance. We Steal Things.,I'm Yours,80,242946,False,0.703,0.4440,...,-9.331,1,0.0417,0.5590,0.000000,0.0973,0.7120,150.960,4,acoustic
8,8,0IktbUcnAGrvD03AWnz3Q8,Jason Mraz;Colbie Caillat,We Sing. We Dance. We Steal Things.,Lucky,74,189613,False,0.625,0.4140,...,-8.700,1,0.0369,0.2940,0.000000,0.1510,0.6690,130.088,4,acoustic
9,9,7k9GuJYLp2AzqokyEdwEw2,Ross Copperman,Hunger,Hunger,56,205594,False,0.442,0.6320,...,-6.770,1,0.0295,0.4260,0.004190,0.0735,0.1960,78.899,4,acoustic


In [ ]:
df.shape

(114000, 21)

In [ ]:
df.dtypes

Unnamed: 0            int64
track_id             object
artists              object
album_name           object
track_name           object
popularity            int64
duration_ms           int64
explicit               bool
danceability        float64
energy              float64
key                   int64
loudness            float64
mode                  int64
speechiness         float64
acousticness        float64
instrumentalness    float64
liveness            float64
valence             float64
tempo               float64
time_signature        int64
track_genre          object
dtype: object

In [ ]:
#Check if have any rows that are null
print(df.isnull().sum())

Unnamed: 0          0
track_id            0
artists             1
album_name          1
track_name          1
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64


In [ ]:
# drop the rows that are null
df = df.dropna()
print(df.isnull().sum())
df.shape

Unnamed: 0          0
track_id            0
artists             0
album_name          0
track_name          0
popularity          0
duration_ms         0
explicit            0
danceability        0
energy              0
key                 0
loudness            0
mode                0
speechiness         0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
time_signature      0
track_genre         0
dtype: int64


(113999, 21)

In [ ]:
df[df.duplicated('track_id')==True]

,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre


In [ ]:
df[df['track_id']=='71dLJx3qHOTQMTvvoE2dmd']


,Unnamed: 0,track_id,artists,album_name,track_name,popularity,duration_ms,explicit,danceability,energy,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,track_genre
80966,4932,71dLJx3qHOTQMTvvoE2dmd,Bethel Music;Amanda Cook,Peace,King of My Heart,48,330840,False,0.515,0.0736,...,-15.492,1,0.031,0.975,0.000261,0.082,0.0684,109.964,4,ambient


In [ ]:
dfDistinct = df.drop_duplicates('track_id')
dfDistinct = dfDistinct.sort_values('track_id')
dfDistinct = dfDistinct.reset_index(drop=True)
df = dfDistinct


In [ ]:
df.shape

(89740, 21)

In [ ]:
df.to_csv('df.csv', index=False)



In [ ]:
df = pd.read_csv('df.csv')


In [ ]:
print(df['track_id'])

0        0000vdREvCVMxbQTkS888c
1        000CC8EParg64OmTxVnZ0p
2        000Iz0K615UepwSJ5z2RE5
3        000RDCYioLteXcutOjeweY
4        000qpdoc97IMTBvF8gwcpy
                  ...          
89735    7zxHiMmVLt4LGWpOMqOpUh
89736    7zxpdh3EqMq2JCkOI0EqcG
89737    7zyYmIdjqqiX6kLryb7QBx
89738    7zybSU9tFO9HNlwmGF7stc
89739    7zz7iNGIWhmfFE7zlXkMma
Name: track_id, Length: 89740, dtype: object


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

# Create a Spotipy object
sp = spotipy.Spotify(auth=access_token)

# Get the track information
track = sp.track('000CC8EParg64OmTxVnZ0p')

# Get the release date
release_date = track['album']['release_date']

# Print the release date
print(release_date)


2021-02-11


In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
def get_release_date(dataset, access_token ):
    sp = spotipy.Spotify(auth=access_token)
    release_dates = []
    for trackid in dataset:
        info = sp.track(trackid)
        time.sleep(2)
        release_date = info['album']['release_date']
        if release_date is not None:
          release_dates.append(release_date)
    return release_dates

In [ ]:
df10= df['track_id'].head(10)
print(df10)

0    0000vdREvCVMxbQTkS888c
1    000CC8EParg64OmTxVnZ0p
2    000Iz0K615UepwSJ5z2RE5
3    000RDCYioLteXcutOjeweY
4    000qpdoc97IMTBvF8gwcpy
5    0017XiMkqbTfF2AUOzlhj6
6    001APMDOl3qtx1526T11n1
7    001YQlnDSduXd5LgBd66gT
8    001pyq8FLNSL1C8orNLI0b
9    002qpSULhHAw6DGqFxbaO1
Name: track_id, dtype: object


In [ ]:
release_dates = get_release_date(df10, access_token)
print(release_dates)

#CALCULATIONS

In [ ]:
# Normalize the music features using Min-Max scaling
scaler = MinMaxScaler()
music_features = df[['danceability', 'energy', 'key', 'mode',
                           'loudness',  'speechiness', 'acousticness',
                           'instrumentalness', 'liveness', 'valence', 'tempo']].values
music_features_scaled = scaler.fit_transform(music_features)

In [ ]:
def content_based_recommendations(input_song_name, num_recommendations=5):
    # Convert the input song name to lowercase.
    input_song_name = input_song_name.lower()

    # Get the indices of the songs in the dataset that match the input song name.
    matching_song_indices = df[df['track_name'].str.lower() == input_song_name].index

    # If no songs match the input song name, return an empty list.
    if not matching_song_indices.any():
        return []

    # Get the index of the input song in the dataset.
    input_song_index = matching_song_indices[0]

    # Calculate the similarity scores based on music features (cosine similarity).
    similarity_scores = cosine_similarity([music_features_scaled[input_song_index]], music_features_scaled)

    # Get the indices of the most similar songs.
    similar_song_indices = similarity_scores.argsort()[0][::-1][1:num_recommendations + 1]

    # Get the names of the most similar songs based on content-based filtering.
    content_based_recommendations = df.iloc[similar_song_indices][['track_name', 'artists', 'album_name', 'track_id', 'popularity']]

    return content_based_recommendations

In [ ]:
# Identify the rows that have the value 'YOASOBI' in the 'artist' column
rows_to_print = df[df['artists'].isin(['YOASOBI'])].index

# Select the identified rows
df_filtered = df.loc[rows_to_print]

# Print the selected rows
print(df_filtered['track_name'])

1197                 怪物
1555              ハルジオン
3681     Into The Night
5315                 群青
7267            Romance
13025             ラブレター
13347             優しい彗星
22928                群青
25759               三原色
29409              ミスター
32593            もう少しだけ
33747                怪物
33789    Into The Night
36262               たぶん
37465               三原色
41919             夜に駆ける
47124             アンコール
48071          あの夢をなぞって
51047         もしも命が描けたら
59858               ハルカ
61635             優しい彗星
63500         もしも命が描けたら
67146          あの夢をなぞって
69450               たぶん
73181             夜に駆ける
79968               ハルカ
81595               好きだ
83966             ハルジオン
84776           Monster
85079                祝福
Name: track_name, dtype: object


In [ ]:
song_name = "lemon"
recommendations = content_based_recommendations(song_name, num_recommendations=5)

# Print the recommendations.
print(f"Content-based recommended songs for '{song_name}':")
print(recommendations)

Content-based recommended songs for 'lemon':
                                   track_name        artists  \
83019                                   Lemon  Kenshi Yonezu   
35808  You're the Inspiration - 2006 Remaster        Chicago   
76899                                Film out            BTS   
76222                                monsters        Camylio   
31250          Please Come Home For Christmas      B.B. King   

                               album_name                track_id  popularity  
83019                               Lemon  7Cd17G3oNQ34OWUwS8ZxfR          53  
35808  Chicago 17 (Expanded & Remastered)  36lkJSDyMEZoWTqtRj8Q8q          73  
76899                       BTS, THE BEST  6g9Vd4z3JnVUiAIRwFlVCG          62  
76222                            monsters  6cQ8IRL1UQR11v4wD4wjo8          67  
31250             Home For Christmas 2022  2i6Zgt3uFaupWIcpvRORMn           0  


In [ ]:
# a function to get hybrid recommendations based on weighted popularity
def hybrid_recommendations(input_song_name, num_recommendations=5, alpha=0.5):

    if input_song_name not in df['track_name'].values:
        print(f"'{input_song_name}' not found in the dataset. Please enter a valid song name.")
        return

    # Get content-based recommendations
    content_based_rec = content_based_recommendations(input_song_name, num_recommendations)

    # Get the popularity score of the input song
    popularity_score = df.loc[df['track_name'] == input_song_name, 'popularity'].values[0]
    print(popularity_score)
    # Calculate the weighted popularity score
    # weighted_popularity_score = popularity_score * calculate_weighted_popularity(df.loc[df['track_name'] == input_song_name, 'Release Date'].values[0])

    # Combine content-based and popularity-based recommendations based on weighted popularity
    hybrid_recommendations = content_based_rec
    hybrid_recommendations = hybrid_recommendations.append({
        'track_name': input_song_name,
        'artists': df.loc[df['track_name'] == input_song_name, 'artists'].values[0],
        'album_name': df.loc[df['track_name'] == input_song_name, 'album_name'].values[0],
        'track_id': df.loc[df['track_name'] == input_song_name, 'track_id'].values[0],
        'popularity':  df.loc[df['track_name'] == input_song_name, 'popularity'].values[0]
    }, ignore_index=True)

    # Sort the hybrid recommendations based on weighted popularity score
    hybrid_recommendations = hybrid_recommendations.sort_values(by='popularity', ascending=False)

    # Remove the input song from the recommendations
    hybrid_recommendations = hybrid_recommendations[hybrid_recommendations['track_name'] != input_song_name]


    return hybrid_recommendations

In [ ]:
song_name = "Lemon"
recommendations = hybrid_recommendations(song_name, num_recommendations=5)

# Print the recommendations.
print(f"Hybrid recommended songs for '{song_name}':")
print(recommendations)

70
Hybrid recommended songs for 'Lemon':
                               track_name    artists  \
1  You're the Inspiration - 2006 Remaster    Chicago   
3                                monsters    Camylio   
2                                Film out        BTS   
4          Please Come Home For Christmas  B.B. King   

                           album_name                track_id  popularity  
1  Chicago 17 (Expanded & Remastered)  36lkJSDyMEZoWTqtRj8Q8q          73  
3                            monsters  6cQ8IRL1UQR11v4wD4wjo8          67  
2                       BTS, THE BEST  6g9Vd4z3JnVUiAIRwFlVCG          62  
4             Home For Christmas 2022  2i6Zgt3uFaupWIcpvRORMn           0  
